<a href="https://colab.research.google.com/github/jespimentel/rag_simplificado/blob/main/rag_simplificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exemplo de RAG
### com Chroma DB e LiteLLM



Baseado nos artigos:
* https://www.nb-data.com/p/simple-rag-implementation-with-contextual
* https://medium.com/@rahultiwari065/unlocking-the-power-of-sentence-embeddings-with-all-minilm-l6-v2-7d6589a5f0aa

---

In [1]:
!pip install -q chromadb pypdf2 sentence-transformers litellm langchain
!pip install -q sympy

In [2]:
import os
import PyPDF2
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import litellm
from litellm import completion
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import Markdown, display
from google.colab import userdata

In [3]:
model = 'deepseek/deepseek-chat'
api_key = userdata.get('DEEPSEEK_KEY')

os.environ['LITELLM_LOG'] = 'DEBUG'
HF_TOKEN = userdata.get('HF_TOKEN')

In [4]:
def extract_text_from_pdfs(folder_path):
    all_text = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                print(f"Lendo o arquivo {filename}")
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    all_text += page.extract_text()
    return all_text

pdf_folder = "/content/drive/MyDrive/pdfs_para_rag"
all_text = extract_text_from_pdfs(pdf_folder)

Lendo o arquivo PVDESF_portugues.pdf
Lendo o arquivo cartilhadireitosmulher_web.pdf
Lendo o arquivo Cartilhaenfrentamento_QRCODE1.pdf
Lendo o arquivo Cartilha-violencia-contra-mulher.pdf
Lendo o arquivo Cartilha_ViolenciaDomestica_18.07.2022.pdf
Lendo o arquivo CARTILHA FORÇA DA REDE.pdf
Lendo o arquivo cartilha-combate-violencia-politica-forum-paulista.pdf


In [5]:
print(len(all_text))

1188944


In [6]:
all_text[6000:7000]

'heres idosas ....................................................................................................................................................................................................... 17\nAs mulheres negras ......................................................................................................................................................................................................................................... 18\nAs mulheres com deficiência ...................................................................................................................................................................................................................... 19\nMulheres imigrantes ........................................................................................................................................................................................................................................ 20\nAs mulheres lésbicas 

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Size of each chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    separators=["\n\n", "\n", " ", ""]  # Splitting hierarchy
)

chunks = text_splitter.split_text(all_text)

In [8]:
print(len(chunks))

2692


In [9]:
print(chunks[50])

WALKER, Lenore E. The battered woman. NY: Harper Perennial, 1979.
2
1
3
Fase 3
L ua de mel
Fase 2
Explosão
Fase 1
Evolução da tensão
Ciclo da Violência
Doméstica
C
M
Y
CM
MY
CY
CMY
KPVDESF_SP_2020-contratoSEBRAE.pdf   12   07/10/2020   15:24:14Ciclo da Violência Doméstica e Familiar contra as Mulheres
Fase 1 – Evolução da T ensão: O parceiro apresenta comportamento 
ameaçador e violento, humilha e xinga a sua parceira, destrói objetos da


In [10]:
# Initialize a persistent ChromaDB client
client = chromadb.PersistentClient(path="/content/drive/MyDrive/pdfs_para_rag/chroma_db")

# Load the SentenceTransformer model for text embeddings
text_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Delete existing collection (if needed)
try:
    client.delete_collection(name="knowledge_base")
    print("Deleted existing collection: knowledge_base")
except Exception as e:
    print(f"Collection does not exist or could not be deleted: {e}")

# Create a new collection for text embeddings
collection = client.create_collection(name="knowledge_base")

# Add text chunks to the collection
for i, chunk in enumerate(chunks):
    # Generate embeddings for the chunk
    embedding = text_embedding_model.encode(chunk)

    # Add to the collection with metadata
    collection.add(
        ids=[f"chunk_{i}"],  # Unique ID for each chunk
        embeddings=[embedding.tolist()],  # Embedding vector
        metadatas=[{"source": "pdf", "chunk_id": i}],  # Metadata
        documents=[chunk]  # Original text
    )

Collection does not exist or could not be deleted: Collection [knowledge_base] does not exists


In [11]:
def semantic_search(query, top_k=5):
    # Generate embedding for the query
    query_embedding = text_embedding_model.encode(query)

    # Query the collection
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
    )
    return results

In [12]:
# Example query
query = "Quais os números dos telefones de emergência?"
results = semantic_search(query)

# Display results
for i, result in enumerate(results['documents'][0]):
    print(f"Result {i+1}:\n{result}\n")
    print("*" * 60 + '\n')

Result 1:
Doméstica sempre que mudar de endereço e telefone – isso viabiliza sua 
proteção pessoal de modo rápido e eficaz, facilitando o acionamento da 
Ronda Maria da Penha, Polícias Civil e Militar em caso de necessidade, além 
auxiliar o andamento das ações judiciais;
• Sobre partilhas de bens, seus direitos serão garantidos; as medidas 
protetivas são temporárias e não implicam em partilha definitiva de bens, 
o que pode ser feito por meio de acordo (mais recomendado) ou processo

************************************************************

Result 2:
CONHEÇA UM POUCO DOS 
SERVIÇOS DA REDE
DELEGACIA ESPECIALIZADA DE
ATENDIMENTO À MULHER-DEAM
MINISTÉRIO PÚBLICO - MP/RN 
(9° PROMOTORIA DE JUSTIÇA DA COMARCA DE MOSSORÓ)
DELEGACIAS DE PLANTÃO
PATRULHA MARIA DA PENHA (Guarda Municipal): 
A patrulha realiza ações que vão desde ações educativas nas 
escolas, órgãos e instituições até o apoio no cumprimento 
de medidas protetivas, garantindo assistência às mulheres. E 
denúncias podem ser

In [13]:
# Set up LiteLLM

def generate_response(query, context):
    # Combine the query and context for the prompt
    prompt = f"Query: {query}\nContext: {context}\nAnswer:"

    # Call the model via LiteLLM
    response = completion(
        model=model,
        messages=[{"content": prompt, "role": "user"}],
        api_key= api_key
    )

    # Extract and return the generated text
    return response['choices'][0]['message']['content']

# Retrieve the top results from semantic search
search_results = semantic_search(query)
context = "\n".join(search_results['documents'][0])

# Generate a response using the retrieved context
response = generate_response(query, context)

print("Generated Response:\n")
display(Markdown(response))

Generated Response:



Aqui estão os números de telefone de emergência mencionados no contexto, especialmente relacionados à proteção de mulheres em situação de violência:

1. **Patrulha Maria da Penha (Guarda Municipal)**  
   - **Telefones:**  
     - **153** (para denúncias e apoio no cumprimento de medidas protetivas)  
     - **(84) 98631-7000** (número direto)  

2. **Delegacias Especializadas de Atendimento à Mulher (DEAM)**  
   - Consulte a delegacia mais próxima em seu município. Em **Mossoró/RN**, a 9ª Promotoria de Justiça pode ser contatada para orientações.  

3. **Números Nacionais de Emergência** (não citados no texto, mas essenciais):  
   - **190** (Polícia Militar)  
   - **180** (Central de Atendimento à Mulher – disque-denúncia anônimo)  
   - **Disque 100** (Direitos Humanos, para violações em geral)  

### Observações:  
- O texto reforça a importância de atualizar seus contatos ao mudar de endereço para garantir acesso rápido a esses serviços.  
- A **Patrulha Maria da Penha** também realiza ações educativas e auxilia no cumprimento de medidas protetivas.  

Caso precise de ajuda imediata, priorize os números **153** (Guarda Municipal) ou **190** (emergência geral).

In [14]:
search_results

{'ids': [['chunk_2578', 'chunk_2625', 'chunk_88', 'chunk_60', 'chunk_1287']],
 'embeddings': None,
 'documents': [['Doméstica sempre que mudar de endereço e telefone – isso viabiliza sua \nproteção pessoal de modo rápido e eficaz, facilitando o acionamento da \nRonda Maria da Penha, Polícias Civil e Militar em caso de necessidade, além \nauxiliar o andamento das ações judiciais;\n• Sobre partilhas de bens, seus direitos serão garantidos; as medidas \nprotetivas são temporárias e não implicam em partilha definitiva de bens, \no que pode ser feito por meio de acordo (mais recomendado) ou processo',
   'CONHEÇA UM POUCO DOS \nSERVIÇOS DA REDE\nDELEGACIA ESPECIALIZADA DE\nATENDIMENTO À MULHER-DEAM\nMINISTÉRIO PÚBLICO - MP/RN \n(9° PROMOTORIA DE JUSTIÇA DA COMARCA DE MOSSORÓ)\nDELEGACIAS DE PLANTÃO\nPATRULHA MARIA DA PENHA (Guarda Municipal): \nA patrulha realiza ações que vão desde ações educativas nas \nescolas, órgãos e instituições até o apoio no cumprimento \nde medidas protetivas, gar

# Outro exemplo

In [15]:
# Example query
query = "O que é violência de gênero?"
results = semantic_search(query)

# Display results
for i, result in enumerate(results['documents'][0]):
    print(f"Result {i+1}:\n{result}\n")
    print("*" * 60 + '\n')

Result 1:
idade, orientação sexual, diversidade de gênero, classe social, 
religião e do vínculo com o agressor.07
APRESENTAÇÃO
será que você sabe 
identificar o que é 
violência?PARA ENTENDERMOS A REDE, 
PRECISAMOS COMPREENDER 
ALGUNS CONCEITOS…
De acordo com o art. 5º da Lei Maria da P enha, violência doméstica 
e familiar contra a mulher é “qualquer ação ou omissão baseada 
no gênero que lhe cause morte, lesão, sofrimento físico, sexual ou

************************************************************

Result 2:
são física, violação sexual,  traumas psicológi -
cos,  danos patrimoniais  e  até mesmo a morte 
de mulheres. Na maioria das vezes é  conside -
rado um crime de ódio, uma vez que este tipo de violência está diretamente relacionado ao 
gênero da vítima.  Ou seja, os atos de violência 
são cometidos contra as mulheres deliberada
-
mente porque são mulheres. 
A partir do momento em que levamos em 
consideração que violência não se trata apenas do ato de agredir ou machucar, pod

In [16]:
# Retrieve the top results from semantic search
search_results = semantic_search(query)
context = "\n".join(search_results['documents'][0])

# Generate a response using the retrieved context
response = generate_response(query, context)

print("Generated Response:\n")
display(Markdown(response))

Generated Response:



**Violência de gênero** é qualquer ação ou omissão baseada no gênero que cause dano físico, sexual, psicológico, patrimonial ou morte, direcionada principalmente contra mulheres e pessoas LGBTQIA+ em razão de sua identidade de gênero, orientação sexual ou posição social. É um crime de ódio, muitas vezes enraizado em desigualdades estruturais e discriminação.  

### Principais características do contexto fornecido:  
1. **Interseccionalidade**: A violência de gênero pode ser agravada por outros fatores como raça, classe social, religião, idade e vínculo com o agressor (como em relações domésticas ou afetivas).  
2. **Formas de violência**:  
   - **Física**: Agressões corporais.  
   - **Psicológica**: Humilhação, controle ou ameaças.  
   - **Sexual**: Estupro, assédio ou coerção.  
   - **Patrimonial**: Destruição ou controle de bens materiais.  
   - **Virtual**: Cyberbullying, difamação ou perseguição online.  
3. **Lei Maria da Penha (Lei nº 11.340/2006)**: Define violência doméstica/familiar contra a mulher e estabelece mecanismos de proteção, incluindo medidas como afastamento do agressor e assistência social.  
4. **Violência política de gênero**: Atinge principalmente mulheres, negros, LGBTQIA+, indígenas e outros grupos marginalizados em espaços de poder, muitas vezes subnotificada devido ao medo ou falta de canais eficazes de denúncia.  

### Como identificar e denunciar:  
- **Sinais**: Comportamento controlador, isolamento social, marcas físicas, mudanças abruptas de comportamento.  
- **Canais**: Disque 180 (Brasil), delegacias especializadas, aplicativos como *Sine Mulher*, ou organizações como o Instituto Marielle Franco para violência política.  

**Resumo**: Violência de gênero é uma manifestação de poder desigual, exigindo respostas legais, sociais e culturais para sua erradicação.

In [17]:
search_results

{'ids': [['chunk_2604',
   'chunk_2431',
   'chunk_2684',
   'chunk_2564',
   'chunk_2672']],
 'embeddings': None,
 'documents': [['idade, orientação sexual, diversidade de gênero, classe social, \nreligião e do vínculo com o agressor.07\nAPRESENTAÇÃO\nserá que você sabe \nidentificar o que é \nviolência?PARA ENTENDERMOS A REDE, \nPRECISAMOS COMPREENDER \nALGUNS CONCEITOS…\nDe acordo com o art. 5º da Lei Maria da P enha, violência doméstica \ne familiar contra a mulher é “qualquer ação ou omissão baseada \nno gênero que lhe cause morte, lesão, sofrimento físico, sexual ou',
   'são física, violação sexual,  traumas psicológi -\ncos,  danos patrimoniais  e  até mesmo a morte \nde mulheres. Na maioria das vezes é  conside -\nrado um crime de ódio, uma vez que este tipo de violência está diretamente relacionado ao \ngênero da vítima.  Ou seja, os atos de violência \nsão cometidos contra as mulheres deliberada\n-\nmente porque são mulheres. \nA partir do momento em que levamos em \nconside